# House price regression

## Description of the notebook

This notebook is organized in 7 parts.

1. Data acquisition: we will import the dataset and get a first glance at what it contains.

2. Dataset exploratory analysis : we will analyze the data, get an understanding of the main features, of their types, if they have missing values...

3. Features exploratory analysis : we will analyze the data,find the corelation between the features and the survival rate and decide which features to keep

4. Data cleaning and feature selection : Select features that will be kept in the model and remove others, deal with missing values if there is any, create dummy variables for categories...

5. Model preparation : prepare the train and test set and prepare the models we will use for the classification

6. Pipeline evaluation and selection: We will run the models and get their scores, which will allow us the choose the best model.

7. Predict : Final stage, we will run our final model to execute predictions.

# 1. Data acquisition

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer

from sklearn.decomposition import PCA

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import VotingRegressor

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

sns.set_style('darkgrid')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
dataset_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

# 2. Dataset Exploratory analysis
In this section we will check the number and types of features for the dataset, if there is missing values, if there is corelation between some features, if we can remove some unnecessary features and if we can create new features from the ones we already have

In [ ]:
def screen_data(df):
    rows = []
    for col in df.columns:
        rows.append([col, df[col].isnull().sum(), df[col].nunique(), df[col].dtypes])
    return pd.DataFrame(rows, columns=['Col', 'Missing values', 'Unique values', 'Type'])

In [ ]:
def screen_missing_values(df, ratio=0.1, verbose=0):
    total_rows = len(df)
    total_columns = len(df.columns)
    df = screen_data(df)
    df_m = df.loc[df['Missing values'] != 0]#.sort_values(['Type', 'Missing values'])
    total_missing_values = len(df_m)
    df_m.loc[:, 'Percentage missing values'] = df_m['Missing values']/total_rows
    missing_values_less10 = len(df_m.loc[df_m['Percentage missing values'] <= ratio])
    missing_values_more10 = len(df_m.loc[df_m['Percentage missing values'] > ratio])
    print('Total missing values : ',total_missing_values, ' --> ', round(100*total_missing_values/total_columns,0), '% of all features')
    print('Columns with less than ', round(ratio*100,0),'% missing values : ',missing_values_less10, ' --> ', round(100*missing_values_less10/total_missing_values, 0), '% of features with missing values')
    print('Columns with more than ', round(ratio*100,0),'% missing values : ',missing_values_more10, ' --> ', round(100*missing_values_more10/total_missing_values, 0), '% of features with missing values')
    if(verbose >= 1):
        print("")
        print(df_m.drop(columns=['Unique values', 'Type']).loc[df_m['Percentage missing values'] > ratio])

## 2.1 Dataframe information

In [ ]:
dataset_train.shape, dataset_test.shape

The train and the test set are the same size, so it's going to be important not to overfit the model.

In [ ]:
dataset_train.head(10)

In [ ]:
df = screen_data(dataset_train)
df.T

### Missing values

In [ ]:
screen_missing_values(dataset_train, verbose=1)

In [ ]:
screen_missing_values(dataset_test, verbose=1)

In the dataset test there is more columns with missing values, but most of them have less than 10% missing values. <br>
The column with more than 10% missing values are the same in train and test. <br>
The columns might have missing values to indicate an absence of pool or fireplace for example, we will have to analyse things a bit more to conclude.

### Types of data

In [ ]:
print('Categorical features : ', len(dataset_train.drop(columns=['Id', 'SalePrice']).select_dtypes(include='object').columns))
print('Numerical features : ', len(dataset_train.drop(columns=['Id', 'SalePrice']).select_dtypes(exclude='object').columns))
print('Total features : ', len(dataset_train.drop(columns=['Id', 'SalePrice']).columns))

A lot of numerical and categorical data

## 2.2 Data description

Based on this very small analysis we can see that we have 79 different features. <br>
In order to start analysing we will have to choose some feature we think might give us a good aproximation of the sale price. <br>
It is not possible to carry an extensive EDA on all the features. <br>
Let's if we can drop some feature which give us the same info (abs(corr) > 0.85), or that represent the same idea.

## Let's check the document describing the data

### Columns we will analyse
- MSZoning because there is probably a difference between an agriculture zone, an industrial zone and a residential zone.
- Lot area, which is the size of the lot
- Neighborhood, probably the price is not the same depending on the neighborhood.
- OverallQual & OverallCond
- YearBuilt & YearRemodAdd
- MoSold, YrSold: maybe the time of year has an impact on the price
- SaleType
- Bedroom & TotRmsAbvGrd
- TotalSF = TotalBsmtSF + 1stFlrSF + 2ndFlrSF

### Columns we will transform
- OverallQual & OverallCond, we can probably aggregate thse columns together
- YearBuilt & YearRemodAdd
- Check if all these columns ExterQual, ExterCond, BsmtCond, BsmtFinType1, BsmtFinType2, HeatingQC, KitchenQual, GarageQual and GarageCond give us a good aproximation of overall quality and overall condition or if there was some errors during the calculation
- TotalSF = TotalBsmtSF + 1stFlrSF + 2ndFlrSF
- TotalBath = FullBath + HalfBath + BsmtFullBath + BsmtHalfBath
- TotalPorch = WoodDeckSF + openPorchSF + EnclosedPorch + 3SsnPorch + ScreenPorch

### Columns we will probably drop
- LandSlope or LandContour, because they contain the same information
- The columns with a lot of missing data, unless they provide very valuable data and we can consider that missing data is just an abscence of pool or fireplace.

# 3. Feature exploratory analysis

Let's start with two categorical data that describe the surroundings of the house MSZoning and Neighborhood.

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(22,5))
sns.boxplot(x="MSZoning", y="SalePrice", data=dataset_train, ax=ax[0])
ax[0].set_title('Price vs MSZoning');
sns.countplot(x="MSZoning", data=dataset_train, ax=ax[1])
ax[1].set_title('Count per MSZoning');

In [ ]:
order = dataset_train.groupby('Neighborhood').agg(np.mean).sort_values('SalePrice').index.to_list()
fig, ax = plt.subplots(2,1, figsize=(22,10))
sns.boxplot(x="Neighborhood", y="SalePrice", data=dataset_train, order=order, ax=ax[0])
ax[0].set_title('Price vs Neighborhood');
sns.countplot(x="Neighborhood", data=dataset_train, order=order, ax=ax[1])
ax[1].set_title('Price vs Neighborhood');

In [ ]:
order = dataset_train.groupby('Neighborhood').agg(np.mean).sort_values('SalePrice').index.to_list()
fig, ax = plt.subplots(1,1, figsize=(22,7))
sns.boxplot(x="Neighborhood", y="SalePrice", hue="MSZoning", data=dataset_train, order=order, ax=ax)
ax.set_title('Price vs Neighborhood');

#### Things we can learn from the previous analysis
- The Neighborhood seems to have a big impact on the price
- Inside a neighborhood, the price can change depending on the zone for example in Crawfor.

### Let's continue witht he size of the house
We will check the lot size and the size of the house.

In [ ]:
dataset_train['TotalSF'] = dataset_train['1stFlrSF'] + dataset_train['2ndFlrSF'] + dataset_train['TotalBsmtSF']
fig, ax = plt.subplots(2,3, figsize=(22,10))
sns.scatterplot(x="LotArea", y="SalePrice", data=dataset_train, ax=ax[0][0])
ax[0][0].set_title('SalePrice vs LotArea');
sns.scatterplot(x="TotalSF", y="SalePrice", data=dataset_train, ax=ax[0][1])
ax[0][1].set_title('SalePrice vs TotalSF');
sns.scatterplot(x="GrLivArea", y="SalePrice", data=dataset_train, ax=ax[0][2])   # LowQualFinSF or BsmtUnfSF
ax[0][2].set_title('SalePrice vs GrLivArea');
sns.scatterplot(x="1stFlrSF", y="SalePrice", data=dataset_train, ax=ax[1][0])
ax[1][0].set_title('SalePrice vs 1stFlrSF');
sns.scatterplot(x="2ndFlrSF", y="SalePrice", data=dataset_train, ax=ax[1][1])
ax[1][1].set_title('SalePrice vs 2ndFlrSF');
sns.scatterplot(x="TotalBsmtSF", y="SalePrice", data=dataset_train, ax=ax[1][2])
ax[1][2].set_title('SalePrice vs TotalBsmtSF');


#### Things we can learn from the previous analysis
- The lot area is not much correlated to the price, but maybe we can transform this column to get a better correlation (log?)
- The 1stFlrSF, 2ndFlrSF and TotalBsmtSF are correlated to the price but show an increase of the variance when the price increases.
- Summing all these values together give us a better correlation, with some outliers that we might have to remove or transform

### Let's check if the overall quality and condition of the house have an impact on the price

In [ ]:
order_qual = dataset_train.groupby('OverallQual').agg(np.mean).sort_values('SalePrice').index.to_list()
order_cond = dataset_train.groupby('OverallCond').agg(np.mean).sort_values('SalePrice').index.to_list()
fig, ax = plt.subplots(2,1, figsize=(22,10))
sns.boxplot(x="OverallQual", y="SalePrice", data=dataset_train, order=order_qual, ax=ax[0])
ax[0].set_title('Price vs OverallQual');
sns.boxplot(x="OverallCond", y="SalePrice", data=dataset_train, order=order_qual, ax=ax[1])
ax[1].set_title('Price vs OverallCond');

#### Things we can learn from the previous analysis
- The overall quality seems a good indicator of the price.
- The overall condition seems less good an indicator of the price, maybe with some binning we could get a better price indicator.

### Let's see what we can learn from the construction year and remodeling year

In [ ]:
#order = dataset_train.groupby('Neighborhood').agg(np.mean).sort_values('SalePrice').index.to_list()
fig, ax = plt.subplots(1,1, figsize=(22,7))
sns.boxplot(x="YearBuilt", y="SalePrice", data=dataset_train, ax=ax)
ax.set_title('Price vs YearBuilt');

In [ ]:
dataset_train['DecadeBuilt'] = dataset_train['YearBuilt']/10
dataset_train['DecadeBuilt'] = dataset_train['DecadeBuilt'].astype(int)*10
#dataset_train[['YearBuilt', 'DecadeBuilt']]

fig, ax = plt.subplots(1,1, figsize=(22,7))
sns.boxplot(x="DecadeBuilt", y="SalePrice", data=dataset_train, ax=ax)
ax.set_title('Price vs YearBuilt');

#### Things we can learn from the previous analysis
- The price of recent houses is a bit higher than houses between 1900 and 1970.
- House in the 1880's and the 1890's are also a bit more expensive.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(22,7))
sns.boxplot(x="YearRemodAdd", y="SalePrice", data=dataset_train, ax=ax)
ax.set_title('Price vs YearRemodAdd');

In [ ]:
dataset_train['DecadeRemodAdd'] = dataset_train['YearRemodAdd']/10
dataset_train['DecadeRemodAdd'] = dataset_train['DecadeRemodAdd'].astype(int)*10
#dataset_train[['YearBuilt', 'DecadeBuilt']]

fig, ax = plt.subplots(1,1, figsize=(22,7))
sns.boxplot(x="DecadeRemodAdd", y="SalePrice", data=dataset_train, ax=ax)
ax.set_title('Price vs DecadeRemodAdd');

#### Things we can learn from the previous analysis
- The price of houses that were remodeled very recently (2010) is also a bit higher than the rest.
- There seem to be a small increase of the price the more recent the remodelation was done


### Let's see what we can learn from the number of room

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(22,7))
sns.boxplot(x="BedroomAbvGr", y="SalePrice", data=dataset_train, ax=ax[0])
ax[0].set_title('Price vs number of bedrooms');
sns.boxplot(x="TotRmsAbvGrd", y="SalePrice", data=dataset_train, ax=ax[1])
ax[1].set_title('Price vs number of rooms');

#### What can we learn from the previous analysis
- The number of rooms is correlated to the price, which was to be expected
- The number of bedroom presents something interesting.
    - A house with 0 rooms above ground has a higher average price than houses with 1, 2 or 3 bedrooms above ground
    - We need to dig a bit more into the houses with 0 bedroom.
        - 1st idea, the number of rooms was badly indicated and we need to estimate it based on other features
        - 2nd idea, the rooms are actually in the basement.

### Let's dig a bit more in the houses with 0 bedroom above ground

In [ ]:
df_0bd = dataset_train.loc[dataset_train['BedroomAbvGr'] == 0]
df_0bd[['YearBuilt', 'BldgType', 'BsmtFinType1', 'BsmtFinType2', 'TotalBsmtSF', 'GrLivArea', 'TotRmsAbvGrd', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'KitchenAbvGr', 'SalePrice']].head(20)

In [ ]:
df_0bd = dataset_test.loc[dataset_train['BedroomAbvGr'] == 0]
df_0bd[['YearBuilt', 'BldgType', 'BsmtFinType1', 'BsmtFinType2', 'TotalBsmtSF', 'GrLivArea', 'TotRmsAbvGrd', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'KitchenAbvGr']].head(20)


Actually there is only a few houses without bedrooms above ground, but some have a very high number of rooms. <br>
There is probably a way of estimating the number of bedrooms

### Let's check if the conditions of the sales and the time of the sale have an impact on the price

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(22,10))
sns.boxplot(x="SaleType", y="SalePrice", data=dataset_train, ax=ax[0][0])
ax[0][0].set_title('Price vs SaleType');
sns.boxplot(x="MoSold", y="SalePrice", data=dataset_train, ax=ax[0][1])
ax[0][1].set_title('Price vs month of sale');
sns.boxplot(x="YrSold", y="SalePrice", data=dataset_train, ax=ax[0][2])
ax[0][2].set_title('Price vs year of sale');
sns.countplot(x="SaleType", data=dataset_train, ax=ax[1][0])
ax[1][0].set_title('Price vs SaleType');
sns.countplot(x="MoSold", data=dataset_train, ax=ax[1][1])
ax[1][1].set_title('Price vs month of sale');
sns.countplot(x="YrSold", data=dataset_train, ax=ax[1][2])
ax[1][2].set_title('Price vs year of sale');

The year and month of sale does not seem to have an impact on the price. <br>
The sale type seems to have an impact but outside of new houses I don't think it is usefull to consider other types of sale because of the population size which is very small.

# 4. Data cleaning and feature selection

Let's start by creating some new columns as we stated in the column analysis.

## 4.1 Feature creation

### Columns we will transform
- OverallQual & OverallCond, we can probably aggregate these columns together
- YearBuilt & YearRemodAdd
- Check if all these columns ExterQual, ExterCond, BsmtCond, BsmtFinType1, BsmtFinType2, HeatingQC, KitchenQual, GarageQual and GarageCond give us a good aproximation of overall quality and overall condition or if there was some errors during the calculation
- TotalSF = TotalBsmtSF + 1stFlrSF + 2ndFlrSF
- TotalBath = FullBath + HalfBath + BsmtFullBath + BsmtHalfBath
- TotalPorch = WoodDeckSF + openPorchSF + EnclosedPorch + 3SsnPorch + ScreenPorch

#### Let's start with a study of the quality and condition of the house

In [ ]:
dataset_train[['OverallQual', 'ExterQual', 'BsmtQual', 'HeatingQC', 'FireplaceQu', 'GarageQual', 'PoolQC']]

In [ ]:
dataset_train[['OverallCond', 'ExterCond', 'BsmtCond', 'HeatingQC', 'GarageCond']]

There is differences of OverallCond between houses with the same score for ExterCond, BsmtCond, HeatingQC, GarageCond.
We could recalculate a new score for the overall condition.

In [ ]:
for col in ['ExterCond', 'BsmtCond', 'HeatingQC', 'GarageCond']:
    dataset_train[col] = dataset_train[col].map({'Ex':9,'Gd':7,'TA':5,'Fa':3,'NA':np.nan}, na_action='ignore')
 
dataset_train['OverallCond_calculated'] = dataset_train[['ExterCond', 'BsmtCond', 'HeatingQC', 'GarageCond']].mean(axis=1)
dataset_train['OverallCond'] = dataset_train['OverallCond'].astype(float)
df = dataset_train.loc[abs(dataset_train['OverallCond'] - dataset_train['OverallCond_calculated']) > 1]
dataset_train['OverallCond_calculated'] = dataset_train['OverallCond_calculated'].astype(int)
dataset_train['OverallCond_calculated_2'] = dataset_train[['OverallCond_calculated', 'OverallCond']].mean(axis=1).astype(int)

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(22,15))
sns.boxplot(x="OverallCond", y="SalePrice", data=dataset_train, ax=ax[0])
ax[0].set_title('Price vs OverallQual');
sns.boxplot(x="OverallCond_calculated", y="SalePrice", data=dataset_train, ax=ax[1])
ax[1].set_title('Price vs OverallCond calculated');
sns.boxplot(x="OverallCond_calculated_2", y="SalePrice", data=dataset_train, ax=ax[2])
ax[2].set_title('Price vs OverallCond calculated 2');

It seems that by averaging the overall condition score and our calculated condition score and binning the result into two categories: <br>
good condition (>=5) and  average condition (<5) we could get a better feature.

#### Let's now check if the calculus of square feet is proprerly made.

In [ ]:
#TotalSF = TotalBsmtSF + 1stFlrSF + 2ndFlrSF + BsmtUnfSF + LowQualFinSF
dataset_train[['TotalSF', 'GrLivArea', 'TotalBsmtSF','1stFlrSF','2ndFlrSF', 'BsmtUnfSF','LowQualFinSF']]

We can see that it doesn't include the basement SF which in my opinion is a mistake, for example on row number 1458, the house has half its surface in the basement. <br>
There must be some value in the surface.

#### Let's see if we can estimathe number of bedrooms

In [ ]:
len(dataset_train.loc[dataset_train['BedroomAbvGr']==0]), len(dataset_test.loc[dataset_test['BedroomAbvGr']==0])

In [ ]:
dataset_train.loc[dataset_train['BedroomAbvGr']==0]

In [ ]:
#df = dataset_train.loc[abs(dataset_train['TotRmsAbvGrd'] - dataset_train['KitchenAbvGr'] -  dataset_train['BedroomAbvGr']) >= 4]
df = dataset_train.copy()
df['bedrooms_ratio'] = dataset_train['BedroomAbvGr']/dataset_train['TotRmsAbvGrd']
df[['TotRmsAbvGrd', 'KitchenAbvGr', 'BedroomAbvGr', 'bedrooms_ratio']].describe()

In [ ]:
df.loc[(df['bedrooms_ratio'] <= 0.2) | (df['bedrooms_ratio'] >= 0.8), ['TotRmsAbvGrd', 'KitchenAbvGr', 'BedroomAbvGr', 'bedrooms_ratio']].head(20)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(22,7))
sns.histplot(x="bedrooms_ratio", data=df, stat="probability", ax=ax[0])
ax[0].set_title('Count of bedrooms_ratio');
sns.scatterplot(x="bedrooms_ratio", y="TotRmsAbvGrd", data=df, ax=ax[1])
ax[1].set_title('TotRmsAbvGrd vs bedrooms ratio');

Let's try to increase the have a bedroom ratio between 0.5 and 0.8 or 0.6 for big houses. <br>
We will increase or decrease the number of bedrooms until we reach a satisfactory ratio.

In [ ]:
mask = (((df['bedrooms_ratio'] < 0.5) & (df['TotRmsAbvGrd'] <= 4)) | ((df['bedrooms_ratio'] <= 0.33) & (df['TotRmsAbvGrd'] >= 5))) | (((df['bedrooms_ratio'] >= 0.7) & (df['TotRmsAbvGrd'] >= 6)) | ((df['bedrooms_ratio'] >= 0.8) & (df['TotRmsAbvGrd'] <= 5)))
df.loc[mask, ['TotRmsAbvGrd', 'KitchenAbvGr', 'BedroomAbvGr', 'bedrooms_ratio']].head(20)

In [ ]:
#Let's first put a kitchen in every house
df.loc[df['KitchenAbvGr'] == 0, 'KitchenAbvGr'] = 1
mask = (((df['bedrooms_ratio'] < 0.5) & (df['TotRmsAbvGrd'] <= 4)) | ((df['bedrooms_ratio'] <= 0.33) & (df['TotRmsAbvGrd'] >= 5))) | (((df['bedrooms_ratio'] >= 0.7) & (df['TotRmsAbvGrd'] >= 5)) | ((df['bedrooms_ratio'] >= 0.8) & (df['TotRmsAbvGrd'] <= 4)))
while len(df.loc[mask]) != 0:
    #small houses, this means that small house have more bedrooms than other room
    df.loc[(df['bedrooms_ratio'] < 0.5) & (df['TotRmsAbvGrd'] <= 4), 'BedroomAbvGr'] += 1
    df.loc[(df['bedrooms_ratio'] >= 0.8) & (df['TotRmsAbvGrd'] <= 4), 'BedroomAbvGr'] -= 1
    #big houses
    df.loc[(df['bedrooms_ratio'] < 0.33) & (df['TotRmsAbvGrd'] >= 5), 'BedroomAbvGr'] += 1
    df.loc[(df['bedrooms_ratio'] >= 0.7) & (df['TotRmsAbvGrd'] >= 5), 'BedroomAbvGr'] -= 1
    #update value of bedroom ratio
    df.loc[:, 'bedrooms_ratio'] = df['BedroomAbvGr']/df['TotRmsAbvGrd']
    mask = (((df['bedrooms_ratio'] < 0.5) & (df['TotRmsAbvGrd'] <= 4)) | ((df['bedrooms_ratio'] <= 0.33) & (df['TotRmsAbvGrd'] >= 5))) | (((df['bedrooms_ratio'] >= 0.7) & (df['TotRmsAbvGrd'] >= 5)) | ((df['bedrooms_ratio'] >= 0.8) & (df['TotRmsAbvGrd'] <= 4)))

In [ ]:
df[['TotRmsAbvGrd', 'KitchenAbvGr', 'BedroomAbvGr', 'bedrooms_ratio']].head(30)

In [ ]:
df[['TotRmsAbvGrd', 'KitchenAbvGr', 'BedroomAbvGr', 'bedrooms_ratio']].describe()

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(22,15))
sns.boxplot(x="BedroomAbvGr", y="SalePrice", data=df, ax=ax[0][0])
ax[0][0].set_title('Price vs number of bedrooms after normalization');
sns.boxplot(x="BedroomAbvGr", y="SalePrice", data=dataset_train, ax=ax[0][1])
ax[0][1].set_title('Price vs number of bedrooms before normalization');
sns.countplot(x="BedroomAbvGr", data=df, ax=ax[1][0])
ax[1][0].set_title('Count of number of bedrooms after normalization');
sns.countplot(x="BedroomAbvGr",data=dataset_train, ax=ax[1][1])
ax[1][1].set_title('Count of number of bedrooms before normalization');

We almost didn't change the distribution of bedrooms in the houses but now we have a more logicalrelation between number of bedrooms and SalePrice.

#### Let's now see if we can normalize the number of rooms and bedrooms by the size of the house.

In [ ]:
dataset_train['SF_per_BedroomAbvGr'] = dataset_train['TotalSF']/dataset_train['BedroomAbvGr']
dataset_train['SF_per_TotRmsAbvGrd_per_sf'] = dataset_train['TotalSF']/dataset_train['TotRmsAbvGrd']
fig, ax = plt.subplots(1,2, figsize=(22,7))
sns.scatterplot(x="SF_per_BedroomAbvGr", y="SalePrice", data=dataset_train, ax=ax[0])
ax[0].set_title('Price vs SF_per_BedroomAbvGr');
sns.scatterplot(x="SF_per_TotRmsAbvGrd_per_sf", y="SalePrice", data=dataset_train, ax=ax[1])
ax[1].set_title('Price vs SF_per_TotRmsAbvGrd_per_sf');

#### Let's check the total number of bath

In [ ]:
dataset_train['Total_bath'] = dataset_train[['FullBath','HalfBath','BsmtFullBath','BsmtHalfBath']].sum(axis=1)
fig, ax = plt.subplots(1,2, figsize=(22,7))
sns.boxplot(x="Total_bath", y="SalePrice", data=dataset_train, ax=ax[0])
ax[0].set_title('Price vs total number of bath');
sns.countplot(x="Total_bath", data=dataset_train, ax=ax[1])
ax[1].set_title('Price vs SF_per_TotRmsAbvGrd_per_sf');

We could categorize the total number of baths with 1, 2, 3 and 4+ number of bath

#### Let's see if the total porch area is a good estimator of the price of the house.

In [ ]:
dataset_train['Total_porch_SF'] = dataset_train[['WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch', 'ScreenPorch']].sum(axis=1)
fig, ax = plt.subplots(1,2, figsize=(22,7))
sns.scatterplot(x="Total_porch_SF", y="SalePrice", data=dataset_train, ax=ax[0])
ax[0].set_title('Price vs Total_porch_SF');

The total size of the porch does not seem to be a good estimator of the SalePrice

#### Let's look at the garage size

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(22,7))
sns.boxplot(x="GarageCars", y="SalePrice", data=dataset_train, ax=ax[0])
ax[0].set_title('Price vs GarageCars');
sns.scatterplot(x="GarageArea", y="SalePrice", data=dataset_train, ax=ax[1])
ax[1].set_title('Price vs GarageArea');
sns.countplot(x="GarageCars", data=dataset_train, ax=ax[2])
ax[2].set_title('Count of GarageCars');

We could do like with Total_bath, 0, 1, 2 or 3+ categories for GarageCars

#### Let's investigate a bit more the feature that don't appear often in the house like pool, fireplace and Misc and their value

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(22,12))
sns.boxplot(x="MiscFeature", y="SalePrice", data=dataset_train, ax=ax[0][0])
ax[0][0].set_title('Price vs MiscFeature');
sns.scatterplot(x="PoolArea", y="SalePrice", data=dataset_train, ax=ax[0][1])
ax[0][1].set_title('Price vs PoolArea');
sns.scatterplot(x="MiscVal", y="SalePrice", data=dataset_train, ax=ax[0][2])
ax[0][2].set_title('SalePrice of MiscVal');
sns.boxplot(x="Fireplaces", y="SalePrice", data=dataset_train, ax=ax[1][0])
ax[1][0].set_title('Price vs Fireplaces');
sns.boxplot(x="FireplaceQu", y="SalePrice", data=dataset_train, ax=ax[1][1])
ax[1][1].set_title('Price vs FireplaceQu');
sns.countplot(x="Fireplaces", data=dataset_train, ax=ax[1][2])
ax[1][2].set_title('Count of Fireplaces');

It does not seem interesting to take into account these Misc features. <br>
Maybe we could integrate the number of fireplaces.

## 4.2 Checking for missing data
For now the features we are considering are : 
- MSZoning, Neighborhood
- OverallQual & OverallCond, ExterCond, BsmtCond, HeatingQC, GarageCond
- YearBuilt & YearRemodAdd
- SaleType
- Bedroom & TotRmsAbvGrd, FullBath + HalfBath + BsmtFullBath + BsmtHalfBath, Kitchen
- TotalSF
- GarageCars


In [ ]:
dataset_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
dataset_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
train = dataset_train[['MSZoning', 'Neighborhood' ,'OverallQual', 'OverallCond', 'ExterCond', 'BsmtCond', 'HeatingQC', 'GarageCond', 'YearBuilt', 'YearRemodAdd', 'SaleType', 'BedroomAbvGr', 'TotRmsAbvGrd', 'FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenAbvGr', 'GrLivArea', 'GarageCars']]
train.isna().sum()

In [ ]:
train.describe()

In [ ]:
test = dataset_test[['MSZoning', 'Neighborhood' ,'OverallQual', 'OverallCond', 'ExterCond', 'BsmtCond', 'HeatingQC', 'GarageCond', 'YearBuilt', 'YearRemodAdd', 'SaleType', 'BedroomAbvGr', 'TotRmsAbvGrd', 'FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenAbvGr', 'GrLivArea', 'GarageCars']]
test.isna().sum()

## 4.3 Checking for similar distribution in the train and test datasets

In [ ]:
def hist_train_test(train, test, col_type="numerical"):
    train['type'] = 'train'
    test['type'] = 'test'
    total = pd.concat([test, train])
    if(col_type=="numerical"):
        cols = total.select_dtypes(include=np.number).columns.to_list()
    if(col_type=="categorical"):
        cols = total.select_dtypes(include=['object', 'category']).columns.to_list()
    nb_rows = int(round(0.9+(len(cols)/3),0))
    i = j = 0
    fig, ax = plt.subplots(nb_rows,3, figsize=(22,nb_rows*6))
    for col in cols:
        if(col_type=="numerical"):
            ax[i][j] = sns.histplot(x=col, data=total, hue="type", multiple="dodge", bins=20, ax=ax[i][j])
        if(col_type=="categorical"):
            ax[i][j] = sns.countplot(x=col, data=total, hue="type", ax=ax[i][j])
        if(j == 2):
            i = i + 1
        j = (j + 1)%3

In [ ]:
hist_train_test(train, test, "numerical")

In [ ]:
hist_train_test(train, test, "categorical")

The distribution of the data between train set and test set is very close. <br>
We don't need to modify our data.

## 4.4 Preprocessing pipeline

Steps to include in the preprocessing pipeline:
- Feature engineering
    - Calculate TotalSF
    - Create a square feet per room and square feet per bedroom feature
    - Create a BuiltDecade and RemodallDecade
    - Recalculate the number of bedrooms
    - MSZoning, put the three smallest categories together in other
    - SaleType, put the smallest categories together in other
    - Total_bath = FullBath + HalfBath + BsmtFullBath + BsmtHalfBath an then we categorize them with 1, 2, 3, or 4+ number of bath
- Impute missing values
    - aaa
- Remove outliers
    - Apply log gtransform to columns
- Encode feature
    - Neighborhood
    - MSZoning
    - SaleType
- Binning of features
    - OverallCond into two categories: good condition (>=5) and  average condition (<5)
    - GarageCars 0, 1, 2, 3+
    - Fireplaces 0, 1, 2+
- Scale features (robust scaler)
- Remove feature (the remaining)

In [ ]:
#Preprocessing functions
def get_TotalSF(df):
    df['TotalSF'] = df['1stFlrSF'] + df['2ndFlrSF'] + df['TotalBsmtSF']
    return df

def get_Total_bath(df):
    df['Total_bath'] = df['FullBath'] + df['HalfBath'] + df['BsmtFullBath'] + df['BsmtHalfBath']
    df.loc[df['Total_bath'] > 4, 'Total_bath'] = 4
    return df

def normalize_SF_per_room(df):
    df['SF_per_BedroomAbvGr'] = df['TotalSF']/df['BedroomAbvGr']
    df['SF_per_TotRmsAbvGrd'] = df['TotalSF']/df['TotRmsAbvGrd']
    return df

def get_decade(df):
    df['DecadeBuilt'] = (df['YearBuilt']/10).astype(int)*10
    df['DecadeRemodAdd'] = (df['YearRemodAdd']/10).astype(int)*10
    return df

def get_estimation_OverallCond(df):
    for col in ['ExterCond', 'BsmtCond', 'HeatingQC', 'GarageCond']:
        df[col] = df[col].map({'Ex':9,'Gd':7,'TA':5,'Fa':3,'NA':np.nan}, na_action='ignore')
    df['OverallCond_calculated'] = df[['ExterCond', 'BsmtCond', 'HeatingQC', 'GarageCond']].mean(axis=1)
    df['OverallCond'] = df['OverallCond'].astype(float)
    df['OverallCond_calculated_2'] = df[['OverallCond_calculated', 'OverallCond']].mean(axis=1).astype(int)
    df.loc[df['OverallCond_calculated_2'] <= 4, 'OverallCond_calculated_2'] = 0
    df.loc[df['OverallCond_calculated_2'] >= 5, 'OverallCond_calculated_2'] = 1
    return df

def get_estimation_bedrooms(df):
    #Let's first put a kitchen in every house
    df.loc[df['KitchenAbvGr'] == 0, 'KitchenAbvGr'] = 1
    df.loc[:, 'bedrooms_ratio'] = df['BedroomAbvGr']/df['TotRmsAbvGrd']
    mask = (((df['bedrooms_ratio'] < 0.5) & (df['TotRmsAbvGrd'] <= 4)) | ((df['bedrooms_ratio'] <= 0.33) & (df['TotRmsAbvGrd'] >= 5))) | (((df['bedrooms_ratio'] >= 0.7) & (df['TotRmsAbvGrd'] >= 5)) | ((df['bedrooms_ratio'] >= 0.8) & (df['TotRmsAbvGrd'] <= 4)))
    while len(df.loc[mask]) != 0:
        #small houses, this means that small house have more bedrooms than other room
        df.loc[(df['bedrooms_ratio'] < 0.5) & (df['TotRmsAbvGrd'] <= 4), 'BedroomAbvGr'] += 1
        df.loc[(df['bedrooms_ratio'] >= 0.8) & (df['TotRmsAbvGrd'] <= 4), 'BedroomAbvGr'] -= 1
        #big houses
        df.loc[(df['bedrooms_ratio'] < 0.33) & (df['TotRmsAbvGrd'] >= 5), 'BedroomAbvGr'] += 1
        df.loc[(df['bedrooms_ratio'] >= 0.7) & (df['TotRmsAbvGrd'] >= 5), 'BedroomAbvGr'] -= 1
        #update value of bedroom ratio
        df.loc[:, 'bedrooms_ratio'] = df['BedroomAbvGr']/df['TotRmsAbvGrd']
        mask = (((df['bedrooms_ratio'] < 0.5) & (df['TotRmsAbvGrd'] <= 4)) | ((df['bedrooms_ratio'] <= 0.33) & (df['TotRmsAbvGrd'] >= 5))) | (((df['bedrooms_ratio'] >= 0.7) & (df['TotRmsAbvGrd'] >= 5)) | ((df['bedrooms_ratio'] >= 0.8) & (df['TotRmsAbvGrd'] <= 4)))
    return df

def bin_MSZoning(df):
    df.loc[(df['MSZoning'] != 'RL') & (df['MSZoning'] != 'RM'), 'MSZoning'] = 'other'
    return df

def bin_SaleType(df):
    df.loc[(df['SaleType'] != 'WD') & (df['SaleType'] != 'new'), 'SaleType'] = 'other'
    return df

def bin_column(df, col, bins, labels):
    df[col] = pd.cut(df[col], bins=bins, labels=labels)
    #df[col] = df[col].astype(float)
    return df

def impute_missing_data(df):
    df.fillna({'MSZoning':df['MSZoning'].mode().iloc[0]}, inplace=True)
    df.fillna({'SaleType':df['SaleType'].mode().iloc[0]}, inplace=True)
    df.fillna({'GarageCars':0, 'BsmtFullBath':0, 'BsmtHalfBath':0, 'TotalBsmtSF':0}, inplace=True)
    return df

def drop_cols(df):
    df = df.drop(df.columns.difference(['MSZoning','SaleType', 'TotalSF', 'Total_bath', 'SF_per_BedroomAbvGr', 'SF_per_TotRmsAbvGrd', 'DecadeBuilt',
            'DecadeRemodAdd', 'OverallCond_calculated_2', 'GarageCars', 'OverallQual', 'Neighborhood', 'bedrooms_ratio', 'BedroomAbvGr', 'TotRmsAbvGrd',
            'KitchenAbvGr', 'Fireplaces']), 1, inplace=True)
    return df

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

combine = [train, test]
for df in combine:
    df = impute_missing_data(df)
    df = get_TotalSF(df)
    df = get_Total_bath(df)
    df = get_estimation_bedrooms(df)
    df = normalize_SF_per_room(df)
    df = get_decade(df)
    df = get_estimation_OverallCond(df)
    df = bin_MSZoning(df)
    df = bin_SaleType(df)
    df = bin_column(df, 'GarageCars', bins=[-0.5,0.5,1.5,2.5,10], labels=[0,1,2,3])
    df = bin_column(df, 'Fireplaces', bins=[-0.5,0.5,1.5,10], labels=[0,1,2])
    df = bin_column(df, 'Total_bath', bins=[-0.5,1.5,2.5,3.5,10], labels=[1,2,3,4])
    df = bin_column(df, 'BedroomAbvGr', bins=[-0.5,2.5,3.5,4.5,20], labels=[2,3,4,5])
    df = bin_column(df, 'TotRmsAbvGrd', bins=[-0.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,20], labels=[3,4,5,6,7,8,9,10])
    df = drop_cols(df)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))         # Sample figsize in inches
sns.heatmap(train.corr(),cmap="BrBG",annot=True, ax=ax)

In [ ]:
train.describe()

In [ ]:
preprocessing = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore', sparse=False) , train.select_dtypes(include="object").columns.to_list()),
    remainder = "passthrough"
)

# 5. Model Preparation

In [ ]:
#dataset_train = pd.read_csv("/kaggle/input/titanic/train.csv")
X = train
y = np.ravel(np.array([pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')['SalePrice']]).T)

X_pred = test.copy()

folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

In [ ]:
pipeline = Pipeline([
    ('preprocessing' , preprocessing),
    #('scaler' ,        MinMaxScaler()),
    ('scaler' ,        StandardScaler()),
    ('pca'    ,        PCA(n_components=10))
])
pipeline.fit(X)
X_preprocessed = pd.DataFrame(pipeline.transform(X))

In [ ]:
X_preprocessed.describe()

In [ ]:
X_pred_preprocessed = pd.DataFrame(pipeline.transform(X_pred))
X_pred_preprocessed.describe()

# 6. Pipeline results

In [ ]:
models_list = { 'LinearRegression': LinearRegression(), 
               'DecisionTree Regressor': DecisionTreeRegressor(),
               'Random Forest': RandomForestRegressor(), 
               'SVR': SVR(), 
               'LGBMRegressor' :LGBMRegressor(verbosity = 0, force_row_wise=True), 
               'XGBRegressor': XGBRegressor(use_label_encoder=False, verbosity = 0), 
               'CatBoostRegressor': CatBoostRegressor(verbose = 0),
               'KernelRidge': KernelRidge(),
               'ElasticNet': ElasticNet(),
               'BayesianRidge': BayesianRidge(),
               'GradientBoostingRegressor': GradientBoostingRegressor()}

model_perf_matrix = []
for model_name, model in models_list.items():
    pipeline = Pipeline([
        ('preprocessing' , preprocessing),
        #('scaler' ,        MinMaxScaler()),
        ('scaler' ,        StandardScaler()),
        ('pca'    ,        PCA(n_components=10)),
        ('model' ,           model)
    ])

    cv_score = cross_val_score(pipeline, X, y, cv=folds, scoring='r2');
    model_perf_matrix.append([model_name, round(cv_score.mean(),3), round(cv_score.std(),4)])
    
df_model_perf = pd.DataFrame(model_perf_matrix, columns=['Model', 'Mean value', 'Std value'])
df_model_perf

In [ ]:
models_list = { 'LinearRegression': { 'model' : LinearRegression(),
                                         'param_grid' : { 
                                                            
                                                         }},
                                'DecisionTreeRegressor': { 'model' : DecisionTreeRegressor(),
                                         'param_grid' : { 
                                                            'min_samples_split': [2,4,10, 15, 20],
                                                            "criterion" : ["mse", "friedman_mse", "mae", "poisson"]
                                                         }},
                                'ElasticNet': { 'model' : ElasticNet(),
                                         'param_grid' : { 
                                                            'alpha' : [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0],
                                                            'l1_ratio' : np.arange(0, 1, 0.01)
                                                         }},
                                'BayesianRidge': { 'model' : BayesianRidge(),
                                         'param_grid' : { 
                                            
                                                         }}}

In [ ]:
%%time

results = {}

for model_name, model in models_list.items():
    best_model = GridSearchCV(estimator=model['model'], param_grid=model['param_grid'], cv=folds)
    best_model.fit(X_preprocessed, y)
    print(model_name)
    print(best_model.best_params_)
    print('Mean score : ', best_model.best_score_, ' Std : ', best_model.cv_results_['std_test_score'][best_model.best_index_])
    model_results = {'estimator' : best_model.best_estimator_, 'best_params' : best_model.best_params_, 
                     'mean' : best_model.best_score_, 'std' : best_model.cv_results_['std_test_score'][best_model.best_index_]}
    results[model_name] = model_results

### The four different models all have very similar scores, let's use all these models in an ensemble classifier

In [ ]:
#create a dictionary of our models
estimators = []
for model_name, model in results.items():
    estimators.append((model_name, model['estimator']))
#create our voting classifier, inputting our models
ensemble = VotingRegressor(estimators)

In [ ]:
ensemble.fit(X_preprocessed, y)

# 7. Prediction

In [ ]:
y_pred = ensemble.predict(X_pred_preprocessed).astype(int)
y_pred

In [ ]:
output = pd.DataFrame({'Id': pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv").Id, 'SalePrice': y_pred})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")